In [1]:
import duckdb

In [2]:
# Import jupysql Jupyter extension to create SQL cells
%load_ext sql 

# Set configrations on jupysql to directly output data to Pandas and to simplify the output that is printed to the notebook.
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:

Deploy Streamlit apps for free on Ploomber Cloud! Learn more: https://ploomber.io/s/signup


In [ ]:
# %%sql
# PRAGMA temp_directory='exp/KA2000//home/cburich_pymnts/buckets/b1/'

,Success


Cargar el dataset:

In [ ]:
dataset_path = '/home/cburich_pymnts/buckets/b1/datasets/'
dataset_file = 'competencia_02_crudisimo.csv'

In [5]:
%%sql

create or replace table competencia_02_crudo as
select
    *
from read_csv_auto("{{dataset_path + dataset_file}}")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


Verificar que todo este bien:

In [6]:
%sql select * from competencia_02_crudo limit 5

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo
0,249221109,201901,1,0,1,59,276,7597.55,47433.58,5654.59,...,0.0,1,0.0,-44919.57,3.23,7136,24336.99,13,0,1466.25
1,249221468,201901,1,0,1,51,13,738.16,-325.09,491.31,...,0.0,1,0.0,-30184.75,15.24,393,16345.79,8,0,1548.36
2,249223005,201901,1,0,1,46,178,1014.31,9434.15,417.36,...,0.0,1,0.0,-3730.14,0.00,2755,1181.65,3,0,3917.82
3,249228180,201901,1,0,1,64,296,1028.33,6873.80,1129.43,...,0.0,1,0.0,-13196.25,0.00,8194,1892.04,1,0,2017.56
4,249232117,201901,1,0,1,77,349,11617.61,43291.55,10726.08,...,0.0,1,0.0,-22073.65,0.00,7876,21902.85,6,0,891.48


In [7]:
%%sql
select
    foto_mes
    , count(*) as cantidad # cuenta cuantos casos hay en cada foto_mes
                           # y lo guarda en un campo llamado cantidad
from competencia_02_crudo
group by foto_mes

,foto_mes,cantidad
0,201901,124752
1,201902,125799
2,201903,126436
3,201904,126996
4,201905,127659
5,201906,129186
6,201907,130724
7,201908,132664
8,201909,134314
9,201910,136682


Crear la clase ternaria:

In [8]:
%%sql

create or replace table competencia_02 as

with periodos as (
    select distinct foto_mes from competencia_02_crudo
), clientes as (
    select distinct numero_de_cliente from competencia_02_crudo
), todo as (
    select numero_de_cliente, foto_mes from clientes cross join periodos
), clase_ternaria as (
    select
        c.*
        , if(c.numero_de_cliente is null, 0, 1) as mes_0 # genero la columna mes_0 y sobre ella genero mes_1 y mes_2
        , lead(mes_0, 1) over (partition by t.numero_de_cliente order by foto_mes) as mes_1
        , lead(mes_0, 2) over (partition by t.numero_de_cliente order by foto_mes) as mes_2
        , if(mes_2 = 1, 'CONTINUA',
             if(mes_1 = 0, 'BAJA+1',
                if(mes_2 = 0,'BAJA+2',NULL))) as clase_ternaria # Replazar null por la lógica que genera el target
    from todo t
    left join competencia_02_crudo c using (numero_de_cliente, foto_mes)
) 

select * EXCLUDE (mes_0, mes_1, mes_2) # descarto las columnas auxiliares
from clase_ternaria
where mes_0 = 1 # descarto las filas de los clientes para los meses previos a que ingresaran al banco.

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


Verificamos:

In [9]:
%sql select count(*) from competencia_02

,count_star()
0,4735593


In [10]:
%%sql
PIVOT competencia_02
on clase_ternaria
USING count(numero_de_cliente)
GROUP BY foto_mes

,foto_mes,BAJA+1,BAJA+2,CONTINUA
0,201901,688,718,123346
1,201902,720,688,124391
2,201903,688,760,124988
3,201904,759,579,125658
4,201905,580,660,126419
5,201906,662,608,127916
6,201907,609,689,129426
7,201908,683,552,131429
8,201909,553,576,133185
9,201910,583,624,135475


In [11]:
%%sql
SELECT
    foto_mes,
    COUNT(CASE WHEN clase_ternaria IS NULL THEN 1 END) AS Nulos,
    COUNT(CASE WHEN clase_ternaria = 'CONTINUA' THEN 1 END) AS CONTINUA,
    COUNT(CASE WHEN clase_ternaria = 'BAJA+1' THEN 1 END) AS "BAJA+1",
    COUNT(CASE WHEN clase_ternaria = 'BAJA+2' THEN 1 END) AS "BAJA+2"
FROM
    competencia_02
GROUP BY
    foto_mes

,foto_mes,Nulos,CONTINUA,BAJA+1,BAJA+2
0,201901,0,123346,688,718
1,201902,0,124391,720,688
2,201903,0,124988,688,760
3,201904,0,125658,759,579
4,201905,0,126419,580,660
5,201906,0,127916,662,608
6,201907,0,129426,609,689
7,201908,0,131429,683,552
8,201909,0,133185,553,576
9,201910,0,135475,583,624


Guardar el dataset modificado:

In [ ]:
%sql COPY competencia_02 TO '{{dataset_path}}competencia_02_ternaria.csv' (FORMAT CSV, HEADER)
# %sql COPY competencia_02 TO '{{dataset_path}}competencia_02.parquet' (FORMAT PARQUET)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success
